In [58]:
import importlib
import GNN
from torch.utils.data import Dataset
import IDS2017_Dataset
importlib.reload(GNN)

<module 'GNN' from 'c:\\TFG\\GNN-NIDS-main\\GNN_NIDS_pytorch\\GNN.py'>

In [59]:
import configparser
import time
import torch
import numpy as np
import os
import tensorflow as tf
from utils import make_or_restore_model
import generator
importlib.reload(generator)
from generator import generate_dataset
import configparser

In [60]:
params = configparser.ConfigParser()
params._interpolation = configparser.ExtendedInterpolation()
params.read('./config.ini')

forceCPU = False

useCuda = torch.cuda.is_available() and not forceCPU
if(useCuda):
    device = torch.cuda.current_device()
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = None
    torch.set_default_tensor_type('torch.FloatTensor')

In [61]:
import preprocess
importlib.reload(preprocess)
from preprocess import sortByTimestamp, checkSorted, shortenDataset
# Aquí es transformen i es posen en format convenient el timestamp del flow, s'ordenen les files segons el temps en ordre ascendent i es treuen files afegides amb capçaleres que hi havia de per mig

# df = sortByTimestamp(os.path.abspath(params["DIRECTORIES"]["train"]), os.path.abspath(params["DIRECTORIES"]["train_sorted"]) + '/train_dataset.csv')

# print(checkSorted(df))

# df.to_csv(os.path.abspath(params["DIRECTORIES"]["train_sorted"]) + '/train_dataset.csv', header=False, index=False)

# df = sortByTimestamp(os.path.abspath(params["DIRECTORIES"]["validation"]), os.path.abspath(params["DIRECTORIES"]["train_sorted"]) + '/train_dataset.csv')

# print(checkSorted(df))

# df.to_csv(os.path.abspath(params["DIRECTORIES"]["validation_sorted"]) + '/eval_dataset.csv', header=False, index=False)

# shortenDataset(os.path.abspath(params["DIRECTORIES"]["train"]), os.path.abspath(params["DIRECTORIES"]["train_short"]) + '/train_dataset.csv')
# shortenDataset(os.path.abspath(params["DIRECTORIES"]["validation"]), os.path.abspath(params["DIRECTORIES"]["validation_short"]) '/eval_dataset.csv')


In [62]:
importlib.reload(IDS2017_Dataset)
train_ds = IDS2017_Dataset.IDS2017_Dataset(os.path.abspath(params["DIRECTORIES"]["train"]))
eval_ds = IDS2017_Dataset.IDS2017_Dataset(os.path.abspath(params["DIRECTORIES"]["validation"]))

In [63]:
# epochs = 2000
epochs = 2000
validation_steps = 600
batch_size = 16
steps_per_epoch = 1600
window = 200
lr = float(params['HYPERPARAMETERS']['learning_rate'])
decay_rate = float(params['HYPERPARAMETERS']['decay_rate'])
decay_steps = int(params['HYPERPARAMETERS']['decay_steps'])

In [64]:
from torch.utils.data import DataLoader

loader_params = {   
                    'batch_size': batch_size,
                    'num_workers': 0
                }
training_generator = DataLoader(train_ds, **loader_params)
evaluating_generator = DataLoader(eval_ds, **loader_params)


In [65]:

model = GNN.GNN(params)
if(useCuda):
    model.to(device=device)
    print(next(model.parameters()).is_cuda)

True


In [66]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
import glob
path = os.path.abspath(params['DIRECTORIES']['logs'])+ "\\ckpt"
files = glob.glob(path + '/*.pt')
if(len(files) > 0):
    files = [(path, path.split('\\')[-1]) for path in files]
    files = [(path, '.'.join(el.split('.')[1:-1])) for (path, el) in files]
    files = [(path, el.split('-')) for (path, el) in files]
    files = [(path, {'Epoch': int(el[0]), 'Loss': float(el[1])}) for (path, el) in files]
    files.sort(key=lambda x: (x[1]['Epoch'], x[1]['Loss']), reverse=True)
    filepath = files[0][0]

    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = Adam(params=model.parameters() , lr=float(params['HYPERPARAMETERS']['learning_rate']))
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loadedEpoch = checkpoint['epoch']
    startingLoss = checkpoint['loss']
    # scheduler = ExponentialLR(optimizer=optimizer, gamma=float(params['HYPERPARAMETERS']['decay_rate']))
    # scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    startingEpoch = loadedEpoch + 1
else:
    startingEpoch = 1
    optimizer = Adam(params=model.parameters() , lr=lr)
    # scheduler = ExponentialLR(optimizer=optimizer, gamma=float(params['HYPERPARAMETERS']['decay_rate']))

In [67]:
from torch.nn import CrossEntropyLoss
import torch
import sys
from sklearn.metrics import f1_score

def train_epoch(gen,epoch_index):
    model.train()
    running_accuracy = 0.0 
    running_loss = 0.
    running_weighted_f1 = 0.0
    running_macro_f1 = 0.0
    total = 0
    loss = CrossEntropyLoss()
    for i, batch in enumerate(gen):
        optimizer.zero_grad(set_to_none=True)
        (input, labels) = batch

        batch_size = labels.size(0)
        window_size = labels.size(1)
        number_classes = labels.size(2)
        output = model(input)

        # The label with the highest value will be our prediction 
        _, predicted = torch.max(output, 2)
        _, labelsIndex = torch.max(labels, 2)
        shape = [batch_size, window_size, 1]
        
        labelsIndexRE = labelsIndex.reshape([batch_size*window_size]).cpu()
        predictedRE = predicted.reshape([batch_size*window_size]).cpu()

        # Adapt input and target for the cross entropy calculation
        output = output.permute([0,2,1])
        labels = labels.permute([0,2,1])

        train_loss = loss(output, labels)
        train_loss.backward()
        optimizer.step()

        running_loss += train_loss.item()
        total += batch_size*window_size
        running_accuracy += (predicted == labelsIndex).sum().item()/(batch_size*window_size)
        running_weighted_f1 += f1_score(labelsIndexRE, predictedRE, average='weighted')
        running_macro_f1 += f1_score(labelsIndexRE, predictedRE, average='macro')

        sys.stdout.write("\r{0}".format("="*int(np.ceil(i/4)) + "{0}>".format("-"*int(np.floor(len(gen)/4)-np.ceil(i/4)))) +
            "    {0}".format(str(i).zfill(3)) + 
            "    {:.5f}".format(running_loss/(i+1)) + "    {:.5f}".format(running_accuracy/(i+1)) + "    {:.5f}".format(running_weighted_f1/(i+1)) + "    {:.5f}".format(running_macro_f1/(i+1)))
        sys.stdout.flush()
    return (running_accuracy/len(gen), running_loss/len(gen), running_weighted_f1/len(gen), running_macro_f1/len(gen), total)

In [68]:
def validate_epoch(gen,epoch_index):
    loss = CrossEntropyLoss()
    running_accuracy = 0.0 
    running_vall_loss = 0.0
    running_weighted_f1 = 0.0
    running_macro_f1 = 0.0
    total = 0
    with torch.no_grad(): 
        model.eval() 
        for i, batch in enumerate(gen):
            (input, labels) = batch
            batch_size = labels.size(0)
            window_size = labels.size(1)
            number_classes = labels.size(2)
            output = model(input)
            
            # # The label with the highest value will be our prediction
            _, predicted = torch.max(output, 2)
            _, labelsIndex = torch.max(labels, 2)

            labelsIndexRE = labelsIndex.reshape([batch_size*window_size]).cpu()
            predictedRE = predicted.reshape([batch_size*window_size]).cpu()

            # Adapt input and target for the cross entropy calculation
            output = output.permute([0,2,1])
            labels = labels.permute([0,2,1])
            eval_loss = loss(output, labels)
            
            running_vall_loss += eval_loss.item()
            total += batch_size*window_size
            running_accuracy += (predicted == labelsIndex).sum().item()/(batch_size*window_size)
            running_weighted_f1 += f1_score(labelsIndexRE, predictedRE, average='weighted')
            running_macro_f1 += f1_score(labelsIndexRE, predictedRE, average='macro')

            sys.stdout.write("\r{0}".format("="*int(np.ceil(i/4)) + "{0}>".format("-"*int(np.floor(len(gen)/4)-np.ceil(i/4)))) +
            "    {0}".format(str(i).zfill(3)) + 
            "    {:.5f}".format(running_vall_loss/(i+1)) + "    {:.5f}".format(running_accuracy/(i+1)) + "    {:.5f}".format(running_weighted_f1/(i+1)) + "    {:.5f}".format(running_macro_f1/(i+1)))
            sys.stdout.flush()
    return (running_accuracy/len(gen), running_vall_loss/len(gen), running_weighted_f1/len(gen), running_macro_f1/len(gen), total)

In [69]:
path_logs = os.path.abspath(params['DIRECTORIES']['logs'])
checkpoint_path=  path_logs + "/ckpt"

In [70]:
for el in model.children():
    print(el)

GRUCell(128, 128)
GRUCell(128, 128)
Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): ReLU()
)
Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): ReLU()
)
Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=64, out_features=15, bias=True)
  (7): Softmax(dim=2)
)


In [71]:
importlib.reload(GNN)
first = True
lastLR = 0.0
for epoch in range(startingEpoch, epochs+1):
    print("\nTraining epoch " + str(epoch))
    (train_acc, train_loss, train_weighted_f1, train_macro_f1, train_total) = train_epoch(training_generator, epoch)
    print("\nValidating epoch " + str(epoch))
    (eval_acc, eval_loss, eval_weighted_f1, eval_macro_f1, eval_total) = validate_epoch(evaluating_generator, epoch)
    thisLR = lr * decay_rate ** (epoch / decay_steps)
    # scheduler.step()
    print("")
    print("     Epoch stats:")
    print("         Last LR: " + str(lastLR))
    print("         This LR: " + str(thisLR))
    lastLR = thisLR
    for g in optimizer.param_groups:
        g['lr'] = thisLR
    print("             TRAIN ----- Loss: " + "{:.2f}".format(train_loss) + " - Accuracy: " + "{:.5f}".format(train_acc)
        + " - Weighted F1: " + "{:.5f}".format(train_weighted_f1) + " - Macro F1: " + "{:.5f}".format(train_macro_f1))
    print("             EVAL  ----- Loss: " + "{:.2f}".format(eval_loss) + " - Accuracy: " + "{:.5f}".format(eval_acc)
        + " - Weighted F1: " + "{:.5f}".format(eval_weighted_f1) + " - Macro F1: " + "{:.5f}".format(eval_macro_f1))
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': eval_loss,
            'accuracy': eval_acc
            }, checkpoint_path + "/weights." + "{0}-".format(str(epoch).zfill(4)) + "{:.2f}".format(eval_loss) + ".pt")


Training epoch 196
===========================================================>    236    1.90177    0.91505    0.89682    0.55427
Validating epoch 196
===============>    058    1.89061    0.92629    0.90752    0.59384
     Epoch stats:
         Last LR: 0.0
         This LR: 0.0009979995670931533
             TRAIN ----- Loss: 1.89 - Accuracy: 0.91120 - Weighted F1: 0.89305 - Macro F1: 0.55194
             EVAL  ----- Loss: 1.86 - Accuracy: 0.91085 - Weighted F1: 0.89239 - Macro F1: 0.58394

Training epoch 197
===========================================================>    236    1.89313    0.92368    0.90512    0.55966
Validating epoch 197
===============>    058    1.88623    0.93064    0.91157    0.59830
     Epoch stats:
         Last LR: 0.0009979995670931533
         This LR: 0.0009979893710702096
             TRAIN ----- Loss: 1.89 - Accuracy: 0.91980 - Weighted F1: 0.90132 - Macro F1: 0.55731
             EVAL  ----- Loss: 1.85 - Accuracy: 0.91513 - Weighted F1: 0.89638 - Ma

KeyboardInterrupt: 